In [1]:
from API_call import API_call
from gamePks import *
import statsapi as mlb
import pandas as pd

from tables import *
import relationships

from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=db.db_engine)
session = Session()

Engine(sqlite://///Users/schlinkertc/code/MLB/mlb.db)


In [2]:
db.db_engine.table_names()

['actions',
 'credits',
 'game_batting_stats',
 'game_pitching_stats',
 'game_player_links',
 'game_team_links',
 'games',
 'hit_data',
 'matchups',
 'pitch_data',
 'pitches',
 'players',
 'plays',
 'runners',
 'team_records',
 'teams',
 'venues']

In [3]:
# Yankees games on my birthday
stmt= """
SELECT 
    pk
FROM
    games
WHERE
    pk 
IN
    (
    SELECT
        gamePk
    FROM
        game_team_links
    WHERE
        team_id 
    IN
        (SELECT
            id
        FROM 
            teams
        WHERE
            name = 'New York Yankees')
    )
AND
    strftime('%m-%d',dateTime)='06-09'
    """

In [4]:
pk = db.db_engine.execute(stmt).first()[0]

In [5]:
call = API_call(pk)

In [16]:
ins = db.insert_game(pk,replace=True)

In [23]:
pd.DataFrame.from_records(ins)

,game,table,insert_status,records_count,records_added,reason
0,565350,games,success,1,1,None
1,565350,plays,success,89,89,None
2,565350,pitches,success,379,379,None
3,565350,pitch_data,success,374,374,None
4,565350,hit_data,success,54,54,None
5,565350,actions,success,45,45,None
6,565350,runners,success,113,113,None
7,565350,credits,success,91,89,None
8,565350,matchups,success,89,89,None
9,565350,venues,success,1,0,None


In [18]:
stmt = f""" 
select 
    *
from 
    game_player_links
where 
    gamePk = {pk}
"""

In [21]:
pd.read_sql(stmt,db.db_engine)

,person_id,team_id,gamePk,gameDateTime,position_code,status_description,gameStatus_isCurrentBatter,gameStatus_isCurrentPitcher,gameStatus_isOnBench,gameStatus_isSubstitute,battingOrder,allPositions
0,595918,None,565350,2019-06-09T17:10:00Z,1,Active,0,0,0,0,None,"1,"
1,669456,None,565350,2019-06-09T17:10:00Z,1,Active,0,0,0,0,None,"1,"
2,592644,None,565350,2019-06-09T17:10:00Z,1,Active,0,0,1,0,None,None
3,641343,None,565350,2019-06-09T17:10:00Z,3,Active,0,0,0,0,800,"3,"
4,621397,None,565350,2019-06-09T17:10:00Z,1,Active,0,0,0,0,None,"1,"
5,621295,None,565350,2019-06-09T17:10:00Z,1,Active,0,0,1,0,None,None
6,547982,None,565350,2019-06-09T17:10:00Z,8,Active,0,0,0,0,900,"8,"
7,608700,None,565350,2019-06-09T17:10:00Z,2,Active,0,0,0,1,902,"11, 2,"
8,605479,None,565350,2019-06-09T17:10:00Z,1,Active,0,0,1,0,None,None
9,502273,None,565350,2019-06-09T17:10:00Z,4,Active,0,0,0,1,401,"4,"


In [8]:
# item for sublist in list for item in sublist

game_stat= list(
    { k for keys in call.game_player_links 
     for k in keys if k.startswith('stats_pitching') }
)

In [9]:
season_stat = list(
    { k for keys in call.game_player_links 
     for k in keys if k.startswith('seasonStats_pitching') }
)

In [10]:
game_stat

['stats_pitching_inheritedRunners',
 'stats_pitching_hits',
 'stats_pitching_doubles',
 'stats_pitching_outs',
 'stats_pitching_gamesPitched',
 'stats_pitching_triples',
 'stats_pitching_gamesPlayed',
 'stats_pitching_holds',
 'stats_pitching_homeRunsPer9',
 'stats_pitching_baseOnBalls',
 'stats_pitching_completeGames',
 'stats_pitching_groundOuts',
 'stats_pitching_intentionalWalks',
 'stats_pitching_runs',
 'stats_pitching_wins',
 'stats_pitching_runsScoredPer9',
 'stats_pitching_earnedRuns',
 'stats_pitching_gamesFinished',
 'stats_pitching_flyOuts',
 'stats_pitching_balls',
 'stats_pitching_sacFlies',
 'stats_pitching_losses',
 'stats_pitching_homeRuns',
 'stats_pitching_caughtStealing',
 'stats_pitching_catchersInterference',
 'stats_pitching_stolenBases',
 'stats_pitching_saveOpportunities',
 'stats_pitching_inningsPitched',
 'stats_pitching_hitBatsmen',
 'stats_pitching_inheritedRunnersScored',
 'stats_pitching_hitByPitch',
 'stats_pitching_atBats',
 'stats_pitching_saves',
 'st

In [11]:
season_stat

['seasonStats_pitching_strikeoutWalkRatio',
 'seasonStats_pitching_outs',
 'seasonStats_pitching_sacBunts',
 'seasonStats_pitching_gamesPlayed',
 'seasonStats_pitching_groundOuts',
 'seasonStats_pitching_earnedRuns',
 'seasonStats_pitching_homeRunsPer9',
 'seasonStats_pitching_wins',
 'seasonStats_pitching_runsScoredPer9',
 'seasonStats_pitching_balks',
 'seasonStats_pitching_baseOnBalls',
 'seasonStats_pitching_losses',
 'seasonStats_pitching_atBats',
 'seasonStats_pitching_strikeoutsPer9Inn',
 'seasonStats_pitching_inheritedRunnersScored',
 'seasonStats_pitching_saves',
 'seasonStats_pitching_walksPer9Inn',
 'seasonStats_pitching_groundOutsToAirouts',
 'seasonStats_pitching_gamesStarted',
 'seasonStats_pitching_runs',
 'seasonStats_pitching_inningsPitched',
 'seasonStats_pitching_sacFlies',
 'seasonStats_pitching_completeGames',
 'seasonStats_pitching_obp',
 'seasonStats_pitching_homeRuns',
 'seasonStats_pitching_caughtStealing',
 'seasonStats_pitching_stolenBases',
 'seasonStats_pit

In [12]:
df = pd.DataFrame.from_records(call.game_player_links)[game_stat]

In [13]:
df[df['stats_batting_stolenBasePercentage']==".---"][['stats_batting_stolenBases','stats_batting_caughtStealing']]


KeyError: 'stats_batting_stolenBasePercentage'

In [ ]:
df[df['stats_batting_stolenBasePercentage'].isna()==True]


In [ ]:
pk = gamePks['2018'][606]

ins = db.insert_game(pk,replace=True)

pd.DataFrame.from_records(ins)

In [ ]:
for table in tables:
    db.db_engine.execute(f'delete from {table} where gamePk={pk}')

In [ ]:
for table in tables:
    r = db.db_engine.execute(f"select count(*) from {table} where gamePk={pk}")
    print(r.fetchone()[0])

In [ ]:
r.fetchone()[0]

In [ ]:
pks = gamePks['2019'][500:550]

calls = [API_call(pk) for pk in pks]
#item for sublist in list for item in sublist
records = [link for call in calls for link in call.game_player_links]

In [ ]:
df = pd.DataFrame.from_records(records)

In [ ]:
df['stats_pitching_gamesPitched'].value_counts()

In [ ]:
df['gameStatus_isCurrentBatter'].value_counts()

In [ ]:
# #TEST: which columns, if any, don't have any values
# dfs = {}
# for table in db.db_engine.table_names():
#     df = pd.read_sql(f"select * from {table}",db.db_engine)
#     dfs[table]=df


# for k in dfs.keys():
#     df = dfs[k]
#     for col in df.columns:
#         if df[col].isna().sum()==df.shape[0]:
#             print(k,col)

# pks = [x[0] for x in session.query(Game.pk).all()]

# calls = [API_call(pk) for pk in pks]

# dfs_fromApi = {}
# for k in calls[0].__dict__.keys():
#     if k[0] != '_':
#         records = [x for call in calls for x in call.__dict__[k]]
#         df = pd.DataFrame.from_records(records)
#         dfs_fromApi[k] = df

In [ ]:
call = API_call(567055)

In [ ]:
liveData = call._result['liveData']

In [ ]:
box_score = liveData['boxscore']

In [ ]:
away = box_score['teams']['away']

In [ ]:
players = away['players']

In [ ]:
player_dicts = []
for k in players.keys():
    player_dicts.append(flatten_dicts(players[k]))



In [ ]:
df = pd.DataFrame.from_records(player_dicts)

In [ ]:
df.info()

In [ ]:
df['status_description'].value_counts()

In [ ]:
df.info(verbose=True)

In [ ]:
stmt = """
select 
    pk,
    count(pk) as count
from
    games
group by 
    pk
having 
    count > 1
"""

In [ ]:
db.db_engine.execute(stmt).fetchall()